In [1]:
# Basic setup 
# Thanks to https://www.youtube.com/@samwitteveenai

!pip -q install langchain huggingface_hub tiktoken
!pip -q install --upgrade together

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
# Verify Langchain version
!pip show langchain

Name: langchain
Version: 0.0.270
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/python/3.10.8/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [4]:
# Check environmental variables, if necessary
import os

# List of avail vars
# os.environ['OPENAI_API_KEY']
# os.environ['HUGGINGFACEHUB_API_TOKEN'] 
# os.environ['GOOGLE_API_KEY'] 
# os.environ['TOGETHER_API_KEY'] 


In [5]:
# Connect to Together.ai for hosted LLM
import together

# set API key
together.api_key = os.environ["TOGETHER_API_KEY"]

print("all done")

all done


In [8]:
# Build a list available models from Together.ai
models = together.Models.list()

# Print full list of available models
for idx, model in enumerate(models):
    print(idx, model['name'])

0 EleutherAI/gpt-j-6b
1 EleutherAI/pythia-12b-v0
2 EleutherAI/pythia-1b-v0
3 EleutherAI/pythia-2.8b-v0
4 EleutherAI/pythia-6.9b
5 HuggingFaceH4/starchat-alpha
6 NousResearch/Nous-Hermes-13b
7 NousResearch/Nous-Hermes-Llama2-13b
8 NumbersStation/nsql-6B
9 OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5
10 OpenAssistant/stablelm-7b-sft-v7-epoch-3
11 bigcode/starcoder
12 databricks/dolly-v2-12b
13 databricks/dolly-v2-3b
14 databricks/dolly-v2-7b
15 google/flan-t5-xl
16 huggyllama/llama-13b
17 huggyllama/llama-30b
18 huggyllama/llama-65b
19 huggyllama/llama-7b
20 lmsys/fastchat-t5-3b-v1.0
21 lmsys/vicuna-13b-v1.3
22 lmsys/vicuna-7b-v1.3
23 prompthero/openjourney
24 runwayml/stable-diffusion-v1-5
25 stabilityai/stable-diffusion-2-1
26 stabilityai/stable-diffusion-xl-base-1.0
27 stabilityai/stablelm-base-alpha-3b
28 stabilityai/stablelm-base-alpha-7b
29 tatsu-lab/alpaca-7b-wdiff
30 timdettmers/guanaco-7b
31 togethercomputer/GPT-JT-6B-v1
32 togethercomputer/GPT-JT-Moderation-6B
33 togethercomp

In [12]:
# Double check the model we want to use
# togethercomputer/llama-2-7b-chat
print(models[51]['name'])


togethercomputer/llama-2-7b-chat


In [13]:
# Start up our model
together.Models.start("togethercomputer/llama-2-7b-chat")

{'success': True,
 'value': '54e8c0ada9901e58698eaa8fbe85a38a5f47c38f6ad228c4a21d38e9fb4e86eb'}

In [21]:
# Import all necessary libraries
import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field

from langchain import PromptTemplate,  LLMChain
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

# Create a container class for our LLM
class TogetherLLM(LLM):
    """Together large language models setup"""

    model: str = "togethercomputer/llama-2-7b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    # class Config:
    #     extra = Extra.forbid

    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text

In [24]:
# Setup sytem message
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""


def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")


def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [25]:
# Instantiate our model with basic params

my_model = "togethercomputer/llama-2-7b-chat"

llm = TogetherLLM(
    model= my_model,
    temperature=0.1,
    max_tokens=512
)

## Basic language translator

In [34]:
# Setup system prompt template
system_prompt = "You are an advanced assistant that excels at translation. "
instruction = "Convert the following text from English to Spanish. Do not repeate the initial text to be translated; simply return the translation:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

[INST]<<SYS>>
You are an advanced assistant that excels at translation. 
<</SYS>>

Convert the following text from English to Spanish. Do not repeate the initial text to be translated; simply return the translation:

 {text}[/INST]


In [35]:
# Create chain for prompt and model
llm_chain = LLMChain(prompt=prompt, llm=llm)


In [36]:
# Let's run a translation request
text = "Where's the bathroom?"
output = llm_chain.run(text)
print(f"English: {text}\nSpanish: {output}")

English: Where's the bathroom?
Spanish:  ¡Dónde está el baño?


## Close down Together model instance

In [37]:
together.Models.stop("togethercomputer/llama-2-7b-chat")

{'success': True}